In [1]:
# Importing required libraries

# !pip install numpy
# !pip install pandas
# !pip install openpyxl
# !pip install plotly-express
# !pip install nbformat==4.2.0
# !pip install ipykernel
# !pip install --upgrade nbformat
# !pip install matplotlib
# !pip install chart-studio

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.figure_factory as ff
import numpy as np   # For numericall array 
import pandas as pd  # For dataframe calculations
from datetime import datetime, timedelta
from scipy.stats import norm


In [2]:
# Input parameters 
Trade_date    = "11/23/2022"
Expire_date   = "05/10/2023"
Spot_price    = 19 # In unit of currency
Strike_price  = 17 # 
Interest_rate = 0.005
Divident_ampl = 0.0
No_of_divide  = 5
Volatility    = 0.3
Covar_yield   = 0

In [3]:
class Pricing:
    """" 
    The code is designed to claclate historical VaR (1 day with .99 confidence level) for the FX portfolio assuming no corrolation between currencies.
    """
    
    def __init__(self, Trade_date, Expire_date, Spot_price, Strike_price, Interest_rate, Divident_ampl, Volatility, Covar_yield, Number_of_payments):

        self.Tr_date             = Trade_date                                                  
        self.Ex_date             = Expire_date                               
        self.Sp_price            = Spot_price                           
        self.St_price            = Strike_price                           	
        self.Ir_rate             = Interest_rate                          
        self.Dv_ampl             = Divident_ampl
        self.Volatality          = Volatility
        self.Covar_yield         = Covar_yield
        self.No_DV_payments      = Number_of_payments

    def Forward_price(self):

        # Convert the input date strings to datetime objects
        start_date = datetime.strptime(self.Tr_date, '%m/%d/%Y')
        end_date = datetime.strptime(self.Ex_date  , '%m/%d/%Y')

        # Calculate the total number of days between the start and end dates
        self.total_days = (end_date - start_date).days

        # Calculate the interval between each generated date
        interval = self.total_days / (self.No_DV_payments + 1)
        
        # Generate the dates
        dates = [start_date + timedelta(days=interval * i) for i in range(1, self.No_DV_payments + 1)]
        
        # Calculate the relative days between the generated dates and the end date
        self.relative_days = [abs((end_date - date).days) / 365 for date in dates]

        self.divident_points = [self.Dv_ampl] * len(self.relative_days)

        # Forward pricing calculation part

        self.F = self.Sp_price*np.exp((self.Ir_rate - self.Covar_yield)*(self.total_days/365))

        for i in range(self.No_DV_payments):

            self.F = self.F - self.divident_points[i]*np.exp(-(self.Ir_rate - self.Covar_yield)*self.relative_days[i])

        return self.F
    

    def d1_and_d2_forward_price(self):

        self.d1 = (np.log(self.F/self.St_price) + (0.5*(self.Volatality**2))*(self.total_days/365))/(self.Volatality*np.sqrt(self.total_days/365)) 

        self.d2 = self.d1 - self.Volatality*np.sqrt(self.total_days/365)

        return self.d1, self.d2


    def d1_and_d2_spot_price(self):

        self.d1 = ((np.log(self.Sp_price/self.St_price)) + (self.Ir_rate +0.5*(self.Volatality**2))*(self.total_days/365))/(self.Volatality*np.sqrt(self.total_days/365))
        
        self.d2 = self.d1 - self.Volatality*np.sqrt(self.total_days/365)

        return self.d1, self.d2
    

    def Call_and_put_from_forward(self):

        self.d1_BL = (np.log(self.F/self.St_price) + (0.5*(self.Volatality**2))*(self.total_days/365))/(self.Volatality*np.sqrt(self.total_days/365)) 

        self.d2_BL = self.d1 - self.Volatality*np.sqrt(self.total_days/365)


        # Call and put price

        self.call = (self.F*norm.cdf(self.d1_BL, loc = 0, scale = 1)   - self.St_price*norm.cdf(self.d2_BL, loc = 0, scale = 1))*np.exp(- self.Ir_rate*(self.total_days/365))

        self.put  = (self.St_price*norm.cdf(-self.d2_BL, loc = 0, scale = 1) - self.F*norm.cdf(-self.d1_BL, loc = 0, scale = 1))*np.exp(- self.Ir_rate*(self.total_days/365))

        return self.call, self.put


    def Call_and_put_from_spot(self):

        self.d1_BL = ((np.log(self.Sp_price/self.St_price)) + (self.Ir_rate + 0.5*self.Volatality**2)*(self.total_days/365))/(self.Volatality*np.sqrt(self.total_days/365))

        self.d2_BL = self.d1 - self.Volatality*np.sqrt(self.total_days/365)

        self.call = norm.cdf(self.d1_BL, loc = 0, scale = 1)*self.Sp_price - norm.cdf(self.d2_BL, loc = 0, scale = 1)*self.St_price*np.exp(- self.Ir_rate*(self.total_days/365))

        self.put  = norm.cdf(-self.d2_BL, loc = 0, scale = 1)*self.St_price*np.exp(- self.Ir_rate*(self.total_days/365)) - norm.cdf(-self.d1_BL, loc = 0, scale = 1)*self.Sp_price

        return self.call, self.put

In [4]:
pricing = Pricing(Trade_date, Expire_date, Spot_price, Strike_price, Interest_rate, Divident_ampl, Volatility, Covar_yield,  No_of_divide)

In [5]:
Dividents = pricing.Forward_price()

print(Dividents)

19.043776380898034


In [6]:
D1, D2 = pricing.d1_and_d2_forward_price()
print("Calculated d1 and d2 from forward price method are: ({:.4f}, {:.4f})".format(D1, D2))

Calculated d1 and d2 from forward price method are: (0.6596, 0.4560)


In [7]:
D1, D2 = pricing.d1_and_d2_spot_price()
print("Calculated d1 and d2 from spot price method are: ({:.4f}, {:.4f})".format(D1, D2))

Calculated d1 and d2 from spot price method are: (0.6596, 0.4560)


In [8]:
call, put = pricing.Call_and_put_from_forward()
print("Calculated d1 and d2 from spot price method are: ({:.4f}, {:.4f})".format(call, put))

Calculated d1 and d2 from spot price method are: (2.6969, 0.6579)


In [9]:
call, put = pricing.Call_and_put_from_spot()
print("Calculated d1 and d2 from spot price method are: ({:.4f}, {:.4f})".format(call, put))

Calculated d1 and d2 from spot price method are: (2.6969, 0.6579)


In [10]:

def unit_test_parity(Trade_date, Expire_date, Spot_price, Strike_price, Interest_rate):

    # end def# Convert the input date strings to datetime objects
    start_date = datetime.strptime(Trade_date, '%m/%d/%Y')
    end_date = datetime.strptime(Expire_date  , '%m/%d/%Y')

    # Calculate the total number of days between the start and end dates
    total_days = (end_date - start_date).days

    time_horozon = ((end_date - start_date).days)/365

    # Parity condition C - P = S - DK

    c_p = float(Spot_price - Strike_price*np.exp(- Interest_rate*time_horozon))

    return  float("{:.2f}".format(c_p))


In [11]:
unit_test_parity(Trade_date, Expire_date, Spot_price, Strike_price, Interest_rate)

2.04